# **Data Cleaning**

Nesta etapa iremos fazer um pré-processamento dos dados, com algumas técnicas de limpeza e transformação iniciais.

- Tratar nulos, descartando registros ausentes ou preenchendo com valores padrão.

- Remover colunas desnecessárias para o projeto.

- Transformar data em dia/mes/ano

- Tratar registros duplicados


---

# # **Importando Pickle do Dataset Original**

In [15]:
import pandas as pd

df = pd.read_pickle('./pickle/df_original.pkl')

df.shape

(5141512, 18)

--- 

# # **Visão Geral do Dataset**

Abaixo, o resultado esperado:

| Coluna                        | Tipo   | Qtd NAs  | Qtd Zeros | Qtd Únicos |
|-------------------------------|--------|----------|-----------|------------|
| Date received                  | object | 0        | 0         | 4534       |
| Product                        | object | 0        | 0         | 21         |
| Sub-product                    | object | 235292   | 0         | 86         |
| Issue                          | object | 2        | 0         | 178        |
| Sub-issue                      | object | 734775   | 0         | 272        |
| Consumer complaint narrative   | object | 3334239  | 0         | 1474741    |
| Company public response        | object | 2679237  | 0         | 11         |
| Company                        | object | 0        | 0         | 7203       |
| State                          | object | 45526    | 0         | 63         |
| ZIP code                       | object | 30225    | 0         | 33623      |
| Tags                           | object | 4659727  | 0         | 3          |
| Consumer consent provided?     | object | 1025256  | 0         | 4          |
| Submitted via                  | object | 0        | 0         | 7          |
| Date sent to company           | object | 0        | 0         | 4483       |
| Company response to consumer   | object | 14       | 0         | 8          |
| Timely response?               | object | 0        | 0         | 2          |
| Consumer disputed?             | object | 4373196  | 0         | 2          |
| Complaint ID                   | int64  | 0        | 0         | 5141512    |


In [16]:
import sys
import os
import pandas as pd

# Adiciona o diretório onde o arquivo functions/describe_dataset.py está localizado
#sys.path.append(os.path.abspath('..'))
#from functions.describe_dataset import analise_descritiva_completa

def analise_descritiva_completa(df):
    resultados = {
        'Coluna': [],
        'Tipo': [],
        'Qtd NAs': [],
        'Qtd Zeros': [],
        'Qtd Únicos': [],  
    }

    for col in df.columns:
        resultados['Coluna'].append(col)
        resultados['Tipo'].append(df[col].dtype)
        resultados['Qtd NAs'].append(df[col].isna().sum())
        resultados['Qtd Zeros'].append((df[col] == 0).sum())
        resultados['Qtd Únicos'].append(df[col].nunique(dropna=True))  
        

    resultados_df = pd.DataFrame(resultados)
    return resultados_df

analise_descritiva_completa(df)

,Coluna,Tipo,Qtd NAs,Qtd Zeros,Qtd Únicos
0,Date received,object,0,0,4534
1,Product,object,0,0,21
2,Sub-product,object,235292,0,86
3,Issue,object,2,0,178
4,Sub-issue,object,734775,0,272
5,Consumer complaint narrative,object,3334239,0,1474741
6,Company public response,object,2679237,0,11
7,Company,object,0,0,7203
8,State,object,45526,0,63
9,ZIP code,object,30225,0,33623


---

# # **Remover Variáveis Desnecessárias**



In [17]:
df =  df.drop(columns=[ 'Date sent to company', 'Submitted via', 'Timely response?', 'Consumer consent provided?'])

df.shape

(5141512, 14)

# # **Remover registros de Empresas pouco relevantes**

Nossa base contém mais de 1000 empresas, porém um grupo menor de empresas concentra a maior parte das reclamações. Vamos remover empresas com menos de 1000 reclamações da nossa análise.



In [18]:
# Contando o número de reclamações por empresa
complaint_counts = df['Company'].value_counts()

# Filtrando para manter apenas empresas com 1000 ou mais reclamações
relevant_companies = complaint_counts[complaint_counts >= 1000].index
df = df[df['Company'].isin(relevant_companies)]

# Verificando o número de registros após o filtro
df_filtered_count = df['Company'].value_counts()

df.shape, df_filtered_count


((4790401, 14),
 Company
 EQUIFAX, INC.                             1064711
 TRANSUNION INTERMEDIATE HOLDINGS, INC.     987780
 Experian Information Solutions Inc.        899810
 BANK OF AMERICA, NATIONAL ASSOCIATION      140173
 WELLS FARGO & COMPANY                      128299
                                            ...   
 TrueAccord Corp.                             1018
 PLANET HOME LENDING, LLC                     1011
 Bull City Financial Solutions, Inc           1007
 Delbert Services                             1006
 Trident Asset Management, L.L.C.             1006
 Name: count, Length: 244, dtype: int64)

# # **Tratar Nulos**

#### Aqui vamos descartar as linhas que contêm valores nulos para as colunas que não conseguimos preencher com valores padrão.

In [19]:
df = df.dropna(subset=['Consumer complaint narrative', 'Sub-issue', 'Sub-product', 'Issue', 'Company public response', 'Company response to consumer', 'State'])

df.shape

(787148, 14)

#### Depois, substituir nulos por algum valor de interesse

- Na coluna Tags, substituir os valores ausentes por 'Not provided'.
- Na coluna Consumer disputed?, substituir os valores ausentes por 'Not provided'.


In [20]:
# Substituir valores ausentes na coluna 'Tags' por 'Not provided'
df.loc[:, 'Tags'] = df['Tags'].fillna('Not provided')

# Substituir valores ausentes na coluna 'Consumer disputed?' por 'Not provided'
df.loc[:, 'Consumer disputed?'] = df['Consumer disputed?'].fillna('Not provided')

# # **Remover Zip Code Inválidos**

Os Zip Code inválidos aparecem com XXXXX

In [21]:
count_XXXXX = (df['ZIP code'] == 'XXXXX').sum()

print(f"Quantidade de registros contendo zip code invalido é: {count_XXXXX}")

Quantidade de registros contendo zip code invalido é: 39745


In [22]:
df = df[df['ZIP code'] != 'XXXXX']

# # **Criar Coluna de Data (Dia, mês, ano)**

In [23]:
df['Date received'] = pd.to_datetime(df['Date received'])

# Extraindo dia, mês e ano
df['Date received Day'] = df['Date received'].dt.day
df['Date received Month'] = df['Date received'].dt.month
df['Date received Year'] = df['Date received'].dt.year

In [24]:
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Company response to consumer',
       'Consumer disputed?', 'Complaint ID', 'Date received Day',
       'Date received Month', 'Date received Year'],
      dtype='object')

# # **Tratar Duplicados**

Nossa análise de reclamações duplicadas indicou cerca de 160.000 registros duplicados. Isso ocorre por vários motivos.

Como tais reclamações podem inclusive conter fraudes, decidimos que seria melhor remove-las do projeto, uma vez que requer uma análise mais aprofundada.

In [25]:
num_duplicates = df['Consumer complaint narrative'].duplicated().sum()
print(f"Número de registros duplicados: {num_duplicates}")


Número de registros duplicados: 157841


## ## **Criar Excel com Duplicados**

A análise das reclamações duplicadas contra bancos diferentes, e com pagamento ou não de indenizações, pode ocorrer devido aos seguintes motivos:

- Modelos de Reclamação Padrão: Alguns consumidores podem usar modelos prontos ou "templates" disponíveis online para fazer suas reclamações. Esses modelos são compartilhados em fóruns, blogs ou por meio de conselheiros legais que fornecem textos sugeridos para facilitar o processo de reclamação. Isso pode resultar em várias reclamações com textos idênticos ou muito semelhantes.

- Serviços de Assistência ao Consumidor: Empresas ou advogados especializados em direitos do consumidor às vezes ajudam várias pessoas a registrar reclamações. Eles podem usar um texto padrão para simplificar o processo, especialmente se lidam com muitos casos semelhantes. Isso pode levar a múltiplas reclamações com o mesmo texto, mas em nome de diferentes consumidores.

- Erro de Processamento ou Importação de Dados: Pode haver um erro no processo de importação ou processamento de dados dentro do sistema que resultou na duplicação de textos de reclamação. Esse tipo de erro técnico pode fazer com que uma mesma reclamação apareça múltiplas vezes, associada a diferentes contas ou empresas.

- Fraude ou Spam: Em alguns casos, pessoas podem enviar múltiplas reclamações com o mesmo texto em uma tentativa de sobrecarregar o sistema ou criar um impacto maior, especialmente se estiverem buscando uma resolução rápida ou compensação.

- Influência de Grupos de Defesa de Direitos: Organizações que defendem os direitos dos consumidores podem incentivar seus membros ou o público a apresentar reclamações sobre certas práticas de forma coordenada, usando um texto padrão para garantir que a mensagem seja clara e consistente.

In [26]:
# Contando as ocorrências de cada valor na coluna 'Consumer complaint narrative'
value_counts = df['Consumer complaint narrative'].value_counts()

# Filtrando os valores com mais de 5 duplicatas
narratives_with_many_duplicates = value_counts[value_counts > 5].index

# Criando o subset do DataFrame com esses registros
df_duplicates = df[df['Consumer complaint narrative'].isin(narratives_with_many_duplicates)]

df_duplicates.to_excel('./excel/df_duplicates.xlsx', index=False)

# # **Amostragem**

Como nas próximas etapas iremos usar APIs que possuem um custo associado, vamos fazer uma amostragem dos dados para reduzir o custo.

Inicialmente vamos trabalhar com uma amostra do dataset, dividindo 50% entre reclamações que geraram pagamento de indenização e 50% que não geraram.

In [27]:
import random

# Valor de 50% da amostra
n_sample = 2500

# 1. Filtrar os registros com 'Closed with monetary relief'
df_monetary_relief = df[df['Company response to consumer'] == 'Closed with monetary relief']

# 2. Filtrar os registros com qualquer outro valor
df_other_responses = df[df['Company response to consumer'] != 'Closed with monetary relief']

# 3. Amostrar 2500 registros de cada grupo
df_monetary_sample = df_monetary_relief.sample(n=n_sample, random_state=42)
df_other_sample = df_other_responses.sample(n=n_sample, random_state=42)

# 4. Combinar as amostras
df_sample = pd.concat([df_monetary_sample, df_other_sample])

# Verificar a distribuição no df_sample
df_sample_distribution = df_sample['Company response to consumer'].value_counts()

df_sample_distribution

(        Date received                      Product  \
 728911     2023-01-15  Credit card or prepaid card   
 3286153    2017-11-07  Credit card or prepaid card   
 1543600    2021-02-24  Checking or savings account   
 3395371    2017-09-13  Checking or savings account   
 4239323    2023-08-26  Checking or savings account   
 
                                         Sub-product  \
 728911   General-purpose credit card or charge card   
 3286153  General-purpose credit card or charge card   
 1543600                             Savings account   
 3395371                            Checking account   
 4239323                            Checking account   
 
                                                      Issue  \
 728911     Problem with a purchase shown on your statement   
 3286153                 Other features, terms, or problems   
 1543600                                 Closing an account   
 3395371  Problem with a lender or other company chargin...   
 4239323       

---

# **Exportando Arquivos Gerados**

In [28]:
df_sample.to_pickle("./pickle/df_cleaned.pkl")

In [2]:
print("Notebook Data Cleaning Concluído")

Notebook Data Cleaning Concluído
